In [117]:
import numpy as np, pandas as pd

In [118]:
!wget https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv

--2019-11-22 09:09:53--  https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.206.128, 2a00:1450:400c:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.206.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [text/csv]
Saving to: ‘bbc-text.csv.1’

bbc-text.csv.1      100%[===================>]   4.82M  4.24MB/s    in 1.1s    

2019-11-22 09:09:56 (4.24 MB/s) - ‘bbc-text.csv.1’ saved [5057493/5057493]



In [437]:
Text=pd.read_csv("bbc-text.csv")
Text.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [462]:
Text.text[0]

'tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being built-in to high

In [463]:
from sklearn.preprocessing import LabelEncoder as le

In [464]:
Text['category']=le().fit_transform(Text['category'])

In [465]:
import string

In [466]:
from nltk.corpus import stopwords

In [467]:
def text_cleaner(message):
    nopunc=[char for char in message if char not in string.punctuation]
    nopunc=''.join(nopunc).split()
    nopunc=[word for word in nopunc if word.lower() not in stopwords.words('english')]
    return nopunc

In [468]:
from sklearn.feature_extraction.text import CountVectorizer

In [503]:
Text_tranformer=CountVectorizer(analyzer=text_cleaner, max_features=400, binary=True).fit(Text['text'])

In [504]:
bow1=Text_tranformer.transform(Text['text'])

In [505]:
bow1.shape

(2225, 400)

In [506]:
print(bow1.toarray())

[[1 0 0 ... 1 1 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [507]:
Data=pd.DataFrame(data=bow1.toarray())
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
1,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [508]:
Data['target']=Text['category']

In [509]:
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,target
0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,4
1,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,3
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [287]:
#import ipdb

In [496]:
class Naive_Bayes(object):
    def __init__(self, features, target):
        self.features=features
        self.target=target
        
    def Phi_ki(self, i, k):
        Res1=np.sum((self.features.iloc[:, i] == 1) & (self.target==k))+1
        Res2=self.target.nunique()
        Res3=np.sum(self.target==k)
        return np.float64(Res1/(Res2+Res3))
    
    def train(self): 
        self.p_y_s=np.array([np.sum(self.target==j)/len(self.target) for j in \
                          range(len(self.target.unique()))])
        
        phi_i_k_s=np.array([[self.Phi_ki(i, k) for k in range(len(self.target.unique()))]\
                        for i in range(self.features.shape[1])])
        self.phi_i_k_s=phi_i_k_s

    def P_Xi_given_y(self, i, k, X):
        return np.float64(self.phi_i_k_s[i][k]**(X[i])*(1-self.phi_i_k_s[i][k])**\
                    (1-X[i]))
        
    def P_Xi(self, i, X):
        return np.sum([self.P_Xi_given_y(i, k, X)*self.p_y_s[k] for k in \
                       range(len(self.target.unique()))])

    
    def P_y_equal_k_given_X(self, k, X):
        Res1=np.prod([self.P_Xi_given_y(i, k, X) for i in range(len(X))])
        Res2=self.p_y_s[k]
        Res3=1.0/np.prod([self.P_Xi(i, X) for i in range(len(X))])
        return Res1*Res2*Res3
    
    def predict(self, X):
        predictions=[]
        for j in range(X.shape[0]):
            if j % 40 == 0:
                print("Prediction progress: {}%".format(100*j/X.shape[0]))
            List=[]
            for k in range(len(self.target.unique())):
                List.append(self.P_y_equal_k_given_X(k, X.iloc[j, :]))
            predictions.append(np.argmax(List))
        print("Prediction progress: {}%".format(100))
        return np.array(predictions)

In [497]:
from sklearn.model_selection import train_test_split

In [510]:
Train_features, Test_features, Train_target, Test_target = train_test_split(Data.iloc[:, :-1],\
        Data.iloc[:, -1], test_size=0.4, random_state=101)

In [511]:
NB=Naive_Bayes(Train_features, Train_target)

In [512]:
NB.train()

In [513]:
Pred=NB.predict(Test_features)

Prediction progress: 0.0%
Prediction progress: 4.49438202247191%
Prediction progress: 8.98876404494382%
Prediction progress: 13.48314606741573%
Prediction progress: 17.97752808988764%
Prediction progress: 22.471910112359552%
Prediction progress: 26.96629213483146%
Prediction progress: 31.46067415730337%
Prediction progress: 35.95505617977528%
Prediction progress: 40.449438202247194%
Prediction progress: 44.943820224719104%
Prediction progress: 49.438202247191015%
Prediction progress: 53.93258426966292%
Prediction progress: 58.42696629213483%
Prediction progress: 62.92134831460674%
Prediction progress: 67.41573033707866%
Prediction progress: 71.91011235955057%
Prediction progress: 76.40449438202248%
Prediction progress: 80.89887640449439%
Prediction progress: 85.3932584269663%
Prediction progress: 89.88764044943821%
Prediction progress: 94.38202247191012%
Prediction progress: 98.87640449438203%
Prediction progress: 100%


In [562]:
accuracy=(np.array(Test_target)==Pred).sum()/len(Test_target)
print("Accuracy of my Bernoulli Naive Bayes: {}%".format(100*accuracy))

Accuracy of my Naive Bayes: 92.02247191011236%


## Comparison with the built-in BernoulliNB

In [521]:
from sklearn.naive_bayes import BernoulliNB as BNB

In [522]:
BernNB=BNB()
BernNB.fit(Train_features, Train_target)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [523]:
BernPred=BernNB.predict(Test_features)

In [564]:
Accuracy=(np.array(Test_target)==BernPred).sum()/len(Test_target)
print("Accuracy of the built-in Bernoulli Naive Bayes: ",Accuracy*100)

Accuracy of the built-in Bernoulli Naive Bayes:  92.02247191011236


## Gaussian Discriminant Analysis

In [554]:
class GDAMulticlass(object):
    
    def __init__(self, features, target, regularizer=1e-50):
        self.features=features
        self.target=target
        self.regularizer=regularizer
        
    def train(self):
        combined=pd.concat([self.features, self.target], axis=1)
        self.mu_s=[combined[combined['target']==j].drop('target', axis=1).mean(axis=0)\
                   for j in range(len(self.target.unique()))]
        self.phi_s=(1.0/len(self.target))*np.array([self.target[self.target==j].count() \
                                    for j in range(len(self.target.unique()))])
        sigma=np.matrix(np.zeros([self.features.shape[1], self.features.shape[1]]))
        for i in range(self.target.shape[0]):
            sigma += np.dot(np.matrix(self.features.iloc[i, :]-\
            self.mu_s[self.target.iloc[i]]).T, \
            np.matrix(self.features.iloc[i, :]-self.mu_s[self.target.iloc[i]]))
        
        self.sigma=(1.0/self.target.shape[0])*sigma
        
    def P_y(self, y, phi_s):
        return phi_s[y]
    
    def P_x_given_y(self, sigma, x, mu):
        if self.features.shape[1]>380:
            
            comp1 = 1.0/(self.regularizer+np.sqrt((2*np.pi)**380) * np.sqrt(\
                                                    np.linalg.det(sigma)))
        else:
            comp1 = 1.0/(np.sqrt((2*np.pi)**self.features.shape[1]) * np.sqrt(\
                                                    np.linalg.det(sigma)))
        comp2 = float(np.exp(np.dot(-0.5*np.dot(x-mu, np.linalg.inv(sigma)), x-mu)))
        return comp1*comp2
        
    def predict(self, X):
        predictions=[]
        for i in range(X.shape[0]):
            Prob=[self.P_x_given_y(self.sigma, X.iloc[i, :], self.mu_s[j])*self.P_y(j,\
                    self.phi_s) for j in range(len(self.target.unique()))]
            
            predictions.append(np.argmax(Prob))
            
        return np.array(predictions)
            

In [555]:
MulticlassGDA=GDAMulticlass(Train_features, Train_target)

In [556]:
MulticlassGDA.train()

In [557]:
GDA_Pred=MulticlassGDA.predict(Test_features)

In [563]:
AccuracyGDA=(np.array(Test_target)==GDA_Pred).sum()/len(Test_target)
print("Accuracy of the GDA: ", AccuracyGDA*100)

Accuracy of the GDA:  91.79775280898876


## Multiclass Logistic Regression